In [5]:
#EDA of dataset
# Load data
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.ensemble import RandomForestClassifier
data1 = pd.read_csv('data1.csv')




In [6]:
print(data1.isnull().sum().sum())  # Total number of NaNs
print(data1['Class'].sum())

0
492


In [7]:


# Define target variable
y = data1['Class']

# Split 70% train, 30% temp (stratified)
train_data, temp_data = train_test_split(data1, test_size=0.3, random_state=43, stratify=y)

# Split temp into 15%/15% (stratified)
test1_data, test2_data = train_test_split(temp_data, test_size=0.5, random_state=43, stratify=temp_data['Class'])

# Check for missing values and class distribution
print(train_data.isnull().sum().sum(), test1_data.isnull().sum().sum(), test2_data.isnull().sum().sum())
print(train_data['Class'].sum(), test1_data['Class'].sum(), test2_data['Class'].sum())


0 0 0
344 74 74


In [ ]:
# Step 2: Prepare SVM training data (legitimate transactions only from train_data)
X_train_nonfraud = train_data[train_data['Class'] == 0].drop(['Class'], axis=1)

# Train OneClassSVM (including 'Time' as a predictor)
svm = OneClassSVM(gamma='auto', nu=0.01, verbose=1)
svm.fit(X_train_nonfraud)

# Step 3: Detect grey area transactions in test1_data
X_test1 = test1_data.drop(['Class'], axis=1)  # Keep 'Time'
svm_pred_test1 = svm.predict(X_test1)  # -1 = anomaly, 1 = normal

# Add predictions to test1_data and filter grey area
test1_data = test1_data.copy()
test1_data['SVM_Flag'] = svm_pred_test1
grey_area = test1_data[test1_data['SVM_Flag'] == -1]
print(len(grey_area))




[LibSVM]

In [ ]:
# Step 4: Prepare RF training data (use original labels)
X_grey = grey_area.drop(['Class', 'SVM_Flag'], axis=1)  # Keep 'Time'
y_grey = grey_area['Class']

# Step 5: Train Random Forest
rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=20,
    class_weight='balanced',
    random_state=42,
    verbose=1
)
rf.fit(X_grey, y_grey)

# Step 6: Predict on test2_data and evaluate
X_test2 = test2_data.drop(['Class'], axis=1)  # Keep 'Time'
y_test2 = test2_data['Class']
rf_pred_test2 = rf.predict_proba(X_test2)[:, 1]


In [ ]:
rf_pred_test2 = (rf_pred_test2 > 0.5).astype(int)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix


# 1. OneClassSVM on Train Set
# svm_pred_train: +1 (normal), -1 (anomaly). Map to 0 (legit), 1 (fraud) for report.
X_train = train_data.drop(['Class'], axis=1)
y_train = train_data['Class']
X_train.isnull().sum().sum(), y_train.isnull().sum().sum()


In [ ]:
labels = [0, 1]
target_names = ['Legitimate', 'Fraud']

svm_pred_train = svm.predict(X_train)
svm_pred_train_labels = np.where(svm_pred_train == 1, 0, 1)


# 1. OneClassSVM on Train Set
print("\n[OneClassSVM] Classification Report on Train Set:")
print(classification_report(y_train, svm_pred_train_labels, target_names=target_names))
print("[OneClassSVM] Confusion Matrix on Train Set:")
print(confusion_matrix(y_train, svm_pred_train_labels, labels=labels))


# 2. OneClassSVM on Test1 Set
y_test1 = test1_data['Class']
svm_pred_test1_labels = np.where(test1_data['SVM_Flag'] == 1, 0, 1)
print("\n[OneClassSVM] Classification Report on Test1 Set:")
print(classification_report(y_test1, svm_pred_test1_labels, target_names=target_names))
print("[OneClassSVM] Confusion Matrix on Test1 Set:")
print(confusion_matrix(y_test1, svm_pred_test1_labels, labels=labels))


# 3. Random Forest on Grey Area Transactions (from Test1)
# y_grey and X_grey already defined
y_grey_pred = rf.predict(X_grey)
print("\n[RandomForest] Classification Report on SVM-Flagged (Grey Area) Transactions (Test1):")
print(classification_report(y_grey, y_grey_pred, target_names=target_names))
print("[RandomForest] Confusion Matrix on SVM-Flagged (Grey Area) Transactions (Test1):")
print(confusion_matrix(y_grey, y_grey_pred, labels=labels))



In [ ]:


# 4. Random Forest on Test2 Set
# y_test2 and rf_pred_test2 already defined
print("\n[RandomForest] Classification Report on Test2 Set:")
print(classification_report(y_test2, rf_pred_test2, target_names=target_names))
print("[RandomForest] Confusion Matrix on Test2 Set:")
print(confusion_matrix(y_test2, rf_pred_test2, labels=labels))

In [ ]:

# 4. Random Forest on Test2 Set
# y_test2 and rf_pred_test2 already defined
print("\n[RandomForest] Classification Report on Test2 Set:")
print(classification_report(y_test2, rf_pred_test2, target_names=target_names))
print("[RandomForest] Confusion Matrix on Test2 Set:")
print(confusion_matrix(y_test2, rf_pred_test2, labels=labels))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Get predicted probabilities for the fraud class (class 1)
y_proba = rf.predict_proba(X_test2)[:, 1]

plt.figure(figsize=(10, 6))
sns.kdeplot(y_proba, bw_adjust=0.5, fill=True, color='royalblue', label='Predicted probability density')

# Mark the default threshold
plt.axvline(0.15, color='red', linestyle='--', label='Default threshold (0.5)')
# Optionally, mark your candidate threshold
plt.axvline(0.3, color='green', linestyle='--', label='Candidate threshold (0.3)')

plt.title('Random Forest Fraud Probability Distribution (Test2 Set)')
plt.xlabel('Predicted Probability of Fraud')
plt.ylabel('Density')
plt.legend()
plt.show()



In [ ]:
plt.figure(figsize=(10, 6))
sns.kdeplot(y_proba[y_test2 == 0], label='Legitimate', fill=True)
sns.kdeplot(y_proba[y_test2 == 1], label='Fraud', fill=True, color='orange')
plt.axvline(0.15, color='red', linestyle='--', label='Default threshold (0.5)')
plt.title('Probability Distribution by True Class')
plt.xlabel('Predicted Probability of Fraud')
plt.ylabel('Density')
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
sns.kdeplot(y_proba[y_test2 == 0], label='Legitimate', fill=True)
sns.kdeplot(y_proba[y_test2 == 1], label='Fraud', fill=True, color='orange')
plt.axvline(0.15, color='red', linestyle='--', label='Default threshold (0.5)')
plt.title('Probability Distribution by True Class')
plt.xlabel('Predicted Probability of Fraud')
plt.ylabel('Density')
plt.ylim(0, 5)  # Set y-axis scale from 0 to 10
plt.legend()
plt.show()



In [ ]:
# Your predictions at default threshold (0.5)
rf_pred_test2 = (y_proba > 0.15).astype(int)
# Find indices of false negatives
false_negatives = (y_test2 == 1) & (rf_pred_test2 == 0)

# Probabilities assigned to these false negatives
fn_probs = y_proba[false_negatives]
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))

# KDE for legitimate
sns.kdeplot(y_proba[y_test2 == 0], label='Legitimate', fill=True)
# KDE for fraud
sns.kdeplot(y_proba[y_test2 == 1], label='Fraud', fill=True, color='orange')
# Plot false negatives as rug or scatter
plt.scatter(fn_probs, [0]*len(fn_probs), color='red', marker='x', label='False Negatives (Fraud Missed)')

# Threshold lines
plt.axvline(0.15, color='red', linestyle='--', label='Default threshold (0.15)')

plt.title('Probability Distribution by True Class\nwith False Negatives Highlighted')
plt.xlabel('Predicted Probability of Fraud')
plt.ylabel('Density')
plt.ylim(0, 5)
plt.legend()
plt.show()


In [ ]:
len(false_negatives)